# Notebook 4: API y Testing

Este notebook cubre el uso de la API REST:
- Iniciar la API FastAPI
- Realizar peticiones HTTP
- Testing de casos de uso
- Integración con aplicaciones
- Benchmarks de rendimiento

**Flujo**: HTTP Request → API → ChromaDB → Response

## 1. Importar Librerías

In [84]:
import requests
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List

## 2. Configuración de la API

In [85]:
API_URL = "http://localhost:8000"

print(f"URL de la API: {API_URL}")
print(f"\nIMPORTANTE: Asegúrate de que la API está corriendo:")
print(f"python3 api_precios.py")

URL de la API: http://localhost:8000

IMPORTANTE: Asegúrate de que la API está corriendo:
python3 api_precios.py


## 3. Verificar que la API está Corriendo

In [86]:
try:
    response = requests.get(f"{API_URL}/")
    if response.status_code == 200:
        data = response.json()
        print(f"API conectada correctamente")
        print(f"Status: {data.get('status')}")
        print(f"Partidas indexadas: {data.get('partidas')}")
    else:
        print(f"Error: {response.status_code}")
except Exception as e:
    print(f"No se pudo conectar a la API")
    print(f"Error: {e}")
    print(f"Inicia la API con: python3 api_precios.py")

Error: 500


## 4. Función Helper para Búsquedas

In [87]:
def buscar_via_api(query: str, n_resultados: int = 5) -> Dict:
    payload = {
        "query": query,
        "n_resultados": n_resultados
    }
    
    response = requests.post(f"{API_URL}/buscar", json=payload)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

def mostrar_resultados(resultado: Dict):
    print(f"Búsqueda: '{resultado['query']}'")
    
    for i, partida in enumerate(resultado['resultados']):
        print(f"\n{i+1}. [{partida['similitud']:.0%}] {partida['concepto']}")
        print(f"   {partida['precio']}€/{partida['unidad']} | {partida['origen']}")
        print(f"   {partida['capitulo']}")
    
    print(f"\nPRECIO SUGERIDO: {resultado['precio_sugerido']}€")
    print(f"Rango: {resultado['precio_min']}€ - {resultado['precio_max']}€")
    print(f"Total indexadas: {resultado['total_indexadas']}")

## 5. Test 1: Búsqueda Básica

In [88]:
resultado1 = buscar_via_api("tubo corrugado M20", n_resultados=5)
mostrar_resultados(resultado1)

Exception: Error 500: Internal Server Error

## 6. Test 2: Búsqueda GET (URL)

In [ ]:
query = "cable RZ1 4x10"
response = requests.get(f"{API_URL}/buscar/{query}?n=3")

if response.status_code == 200:
    resultado2 = response.json()
    mostrar_resultados(resultado2)
else:
    print(f"Error: {response.status_code}")

Búsqueda: 'cable RZ1 4x10'

1. [76%] Cable RZ1-K 3x4mm2 libre de halógenos para circuitos de fuerza
   2.15€/m | presupuesto_vivienda
   INSTALACIÓN ELÉCTRICA

2. [74%] Cable RZ1-K 3x2.5mm2 libre de halógenos para circuitos de alumbrado
   1.35€/m | presupuesto_vivienda
   INSTALACIÓN ELÉCTRICA

3. [66%] Cable coaxial RG-6 para TV terrestre y satélite
   0.75€/m | presupuesto_vivienda
   INSTALACIÓN DE TELECOMUNICACIONES

PRECIO SUGERIDO: 1.42€
Rango: 0.75€ - 2.15€
Total indexadas: 24


## 7. Test 3: Múltiples Búsquedas

In [ ]:
queries_test = [
    "instalación eléctrica vivienda",
    "cuadro protección",
    "mecanismo conmutador",
    "punto de luz simple",
    "bandeja portacables"
]

resultados_test = []

for query in queries_test:
    try:
        resultado = buscar_via_api(query, n_resultados=3)
        resultados_test.append({
            "query": query,
            "precio_sugerido": resultado['precio_sugerido'],
            "num_resultados": len(resultado['resultados']),
            "similitud_media": sum(r['similitud'] for r in resultado['resultados']) / len(resultado['resultados'])
        })
        print(f"{query}: {resultado['precio_sugerido']}€")
    except Exception as e:
        print(f"Error en '{query}': {e}")

df_test = pd.DataFrame(resultados_test)
print("\nResumen de tests:")
display(df_test)

instalación eléctrica vivienda: 311.32€
cuadro protección: 109.15€
mecanismo conmutador: 2.78€
punto de luz simple: 2.78€
bandeja portacables: 4.4€

Resumen de tests:


,query,precio_sugerido,num_resultados,similitud_media
0,instalación eléctrica vivienda,311.32,3,0.488667
1,cuadro protección,109.15,3,0.330000
2,mecanismo conmutador,2.78,3,0.471000
3,punto de luz simple,2.78,3,0.238333
4,bandeja portacables,4.40,3,0.455333


## 9. Test 5: Caso de Uso Real - Presupuestar Vivienda

In [ ]:
print("CASO DE USO: Presupuesto de instalación eléctrica\n")

partidas_vivienda = [
    {"concepto": "Instalación eléctrica completa vivienda", "cantidad": 1, "unidad": "vivienda"},
    {"concepto": "Cuadro general protección", "cantidad": 1, "unidad": "ud"},
    {"concepto": "Punto de luz simple", "cantidad": 15, "unidad": "ud"},
    {"concepto": "Base enchufe 16A", "cantidad": 20, "unidad": "ud"},
    {"concepto": "Mecanismo conmutador", "cantidad": 5, "unidad": "ud"}
]

presupuesto_total = 0
detalles_presupuesto = []

for item in partidas_vivienda:
    resultado = buscar_via_api(item['concepto'], n_resultados=1)
    precio_unitario = resultado['precio_sugerido']
    importe = precio_unitario * item['cantidad']
    presupuesto_total += importe
    
    detalles_presupuesto.append({
        "Concepto": item['concepto'],
        "Cantidad": item['cantidad'],
        "Unidad": item['unidad'],
        "Precio Unit.": f"{precio_unitario:.2f}€",
        "Importe": f"{importe:.2f}€"
    })
    
    print(f"{item['cantidad']:3} {item['unidad']:10} {item['concepto']:40} "
          f"{precio_unitario:8.2f}€  {importe:10.2f}€")

print(f"\n{'TOTAL PRESUPUESTO':70} {presupuesto_total:10.2f}€")

df_presupuesto = pd.DataFrame(detalles_presupuesto)
print("\nPresupuesto detallado:")
display(df_presupuesto)

CASO DE USO: Presupuesto de instalación eléctrica

  1 vivienda   Instalación eléctrica completa vivienda    245.50€      245.50€
  1 ud         Cuadro general protección                   78.50€       78.50€
 15 ud         Punto de luz simple                          3.45€       51.75€
 20 ud         Base enchufe 16A                             4.25€       85.00€
  5 ud         Mecanismo conmutador                         4.25€       21.25€

TOTAL PRESUPUESTO                                                          482.00€

Presupuesto detallado:


,Concepto,Cantidad,Unidad,Precio Unit.,Importe
0,Instalación eléctrica completa vivienda,1,vivienda,245.50€,245.50€
1,Cuadro general protección,1,ud,78.50€,78.50€
2,Punto de luz simple,15,ud,3.45€,51.75€
3,Base enchufe 16A,20,ud,4.25€,85.00€
4,Mecanismo conmutador,5,ud,4.25€,21.25€


## 10. Test 6: Validación de Respuestas

In [ ]:
def validar_respuesta(resultado: Dict) -> bool:
    campos_requeridos = [
        'query', 'resultados', 'precio_sugerido', 
        'precio_min', 'precio_max', 'total_indexadas'
    ]
    
    for campo in campos_requeridos:
        if campo not in resultado:
            print(f"Falta campo: {campo}")
            return False
    
    if not isinstance(resultado['resultados'], list):
        print(f"'resultados' no es una lista")
        return False
    
    if len(resultado['resultados']) > 0:
        campos_partida = ['concepto', 'precio', 'unidad', 'similitud', 'origen', 'capitulo']
        for campo in campos_partida:
            if campo not in resultado['resultados'][0]:
                print(f"Falta campo en partida: {campo}")
                return False
    
    return True

test_queries = ["tubo corrugado", "cable", "mecanismo"]

for query in test_queries:
    resultado = buscar_via_api(query, n_resultados=3)
    valido = validar_respuesta(resultado)
    status = "OK" if valido else "ERROR"
    print(f"{status} - '{query}'")

OK - 'tubo corrugado'
OK - 'cable'
OK - 'mecanismo'


## 11. Ejemplo de Integración Python

In [ ]:
class PropaPherClient:
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url
    
    def buscar(self, query: str, n_resultados: int = 5) -> Dict:
        response = requests.post(
            f"{self.base_url}/buscar",
            json={"query": query, "n_resultados": n_resultados}
        )
        response.raise_for_status()
        return response.json()
    
    def obtener_precio_sugerido(self, query: str) -> float:
        resultado = self.buscar(query, n_resultados=5)
        return resultado['precio_sugerido']
    
    def health_check(self) -> bool:
        try:
            response = requests.get(f"{self.base_url}/")
            return response.status_code == 200
        except:
            return False

client = PropaPherClient()

if client.health_check():
    print("Cliente conectado")
    precio = client.obtener_precio_sugerido("cable RZ1")
    print(f"Precio cable RZ1: {precio:.2f}€")
else:
    print("No se pudo conectar al servidor")

Cliente conectado
Precio cable RZ1: 2.82€


## 12. Documentación de la API

In [ ]:
print("""
DOCUMENTACIÓN DE LA API
═══════════════════════════════════════════════

Base URL: http://localhost:8000

Endpoints:

1. GET /
   Descripción: Health check y estado del sistema
   Respuesta: {"status": "ok", "partidas": 1234}

2. POST /buscar
   Descripción: Búsqueda semántica de partidas
   Body: {
     "query": "tubo corrugado M20",
     "n_resultados": 5
   }
   Respuesta: {
     "query": "...",
     "resultados": [...],
     "precio_sugerido": 12.50,
     "precio_min": 10.00,
     "precio_max": 15.00,
     "total_indexadas": 1234
   }

3. GET /buscar/{query}?n={num}
   Descripción: Búsqueda vía URL
   Ejemplo: /buscar/tubo corrugado?n=3
   Respuesta: Igual que POST /buscar

Documentación interactiva:
   http://localhost:8000/docs (Swagger UI)
   http://localhost:8000/redoc (ReDoc)
""")


DOCUMENTACIÓN DE LA API
═══════════════════════════════════════════════

Base URL: http://localhost:8000

Endpoints:

1. GET /
   Descripción: Health check y estado del sistema
   Respuesta: {"status": "ok", "partidas": 1234}

2. POST /buscar
   Descripción: Búsqueda semántica de partidas
   Body: {
     "query": "tubo corrugado M20",
     "n_resultados": 5
   }
   Respuesta: {
     "query": "...",
     "resultados": [...],
     "precio_sugerido": 12.50,
     "precio_min": 10.00,
     "precio_max": 15.00,
     "total_indexadas": 1234
   }

3. GET /buscar/{query}?n={num}
   Descripción: Búsqueda vía URL
   Ejemplo: /buscar/tubo corrugado?n=3
   Respuesta: Igual que POST /buscar

Documentación interactiva:
   http://localhost:8000/docs (Swagger UI)
   http://localhost:8000/redoc (ReDoc)

